### Code to Calculate Free Energy of Releasing Distance Restraint

In [26]:
# Code closely based on restraints.py in
# Yank https://github.com/choderalab/yank 
# (for releasing Boresch restraints)

import numpy as np
import os
from math import pi, sin, log
import scipy.integrate
from Sire import Units

# Constants
v0 = ((Units.meter3/1000)/Units.mole.value()).value() # A^3, the standard state volume
R = Units.gasr # kcal mol-1, the molar gas constant
T = 298.15 # K

def numerical_distance_integrand(r, r0, r_fb, kr):
    """Integrand for harmonic distance restraint. Domain is on [0, infinity], 
    but this will be truncated to [0, 8 RT] for practicality.

    Args:
        r (float): Distance to be integrated, in Angstrom 
        r0 (float): Equilibrium distance, in Angstrom
        r_fb (float): Flat-bottomed radius, in Angstrom
        kr (float): Force constant, in kcal mol-1 A-2

    Returns:
        float: Value of integrand
    """
    r_eff = abs(r - r0) - r_fb
    if r_eff < 0:
        r_eff = 0
    return (r**2)*np.exp(-(kr*r_eff**2)/(2*R*T))

def get_correction(r0, r_fb, kr):
    """Get the free energy of releasing the harmonic distance restraint.
    Domain is on [0, infinity], but this will be truncated to [0, 8 RT] for practicality.
    Args:
        r0 (float): Equilibrium distance, in Angstrom
        r_fb (float): Flat-bottomed radius, in Angstrom
        kr (float): Force constant, in kcal mol-1 A-2
    Returns:
        float: Free energy of releasing the restraint
    """
    dist_at_8RT = 4*np.sqrt((R*T)/kr) + fb_radius # Dist. which gives restraint energy = 8 RT
    r_min = max(0, r0-dist_at_8RT)
    r_max = r0 + dist_at_8RT
    integrand = lambda r: numerical_distance_integrand(r, r0, r_fb, kr)
    z_r = scipy.integrate.quad(integrand, r_min, r_max)[0]
    dg = -R*T*log(v0/(4*np.pi*z_r))

    return dg

### Example: Free Energy of Releasing Restraint for M-Hand-R

Here, r0 = 2.72 A, the force constant is 40 kcal mol-1 A-2, and the flat-bottomed radius is 0.61 A

In [28]:
cor = get_correction(2.72, 0.61, 40)
print(f"Free energy of releasing flat-bottomed restraint: {cor:.2f} kcal mol-1")

Free energy of releasing flat-bottomed restraint: -1.44 kcal mol-1
